# Analyzing cappable-seq pilot experiment

## Pipeline

Similar to the `dusp11_clip_seq` pipeline:
1. Trim any adapter sequences using `cutadapt`
2. Deplete any ribosomal RNA using `SortMeRNA`
3. Align to hybrid WSN/human genome index using `STAR`
4. Quantify transcripts using `Salmon`
5. Measure differential expression using `DeSeq2`

## Dealing with mycoplasma contamination

I processed this pipeline to the quantification step and had frustratingly low alignment percentages at both the genomic and transcriptomic levels. I BLAST'ed some of the unmapped reads and found mycoplasma contamination in all samples. Bad. To remove as many myco reads as possible, I'm including the myco_genome.fasta along with human_rRNAs.fasta during ribodepletion to pull out myco sequences as well.

## Make `filename-utils.py` module to allow repeated extraction of R1/R2 reads

Sequencing core ran paired-end sequencing and returned foreward/reverse files for each sample (R1/R2). I'm going to make a script that will iterate over a folder and return each file in a dict with the sample name so samplename_R1 and samplename_R2 will be in a dict{samplename: [samplename_R1, samplename_R2]}. This can be called in subsequent files as a function to find all read files in a directory for a given program to run them.

In [7]:
# %load filename_utils.py - NOTE: this line imports the text of the file into this cell, 
# is not included in the actual file
#! /usr/bin/env python3
__name__ = "filename_utils"
__author__ = "Thomas"
__date__ = "2024-05-08" 

import os

# Assumes prefixes are the same for read files with _R1 and _R2 denoting fwd and rev reads

def get_filenames(directory):
    replicates = {}

    for dirpath, dirnames, filenames in os.walk(directory):
        for file in filenames:
            if ".log" not in file:
                name = file.split("_R")[0]
                if name not in replicates:
                    replicates[name] = []
                replicates[name].append(file)

    return replicates

## Ribodeplete
We shouldn't have many rRNA due to ribodepletion of inputs and the nature of cappable-seq samples, but will still use `sortmeRNA` to align to human rRNA sequences and see what comes out. (NOTE: also myco sequences in combined .fasta file.)

In [2]:
# %load run_sortmeRNA.py
import os
import subprocess
import shutil
from filename_utils import get_filenames

basedir=os.path.abspath(os.path.dirname(__file__))
PUID = os.getuid()
PGID = os.getgid()
trimmed_reads_dir = f"{basedir}/trimmed_reads" #Changed for myco removal
ribofile = f"{basedir}/decon_reads.fasta"
sort_dir = f"{basedir}/ribodepleted_reads"

if not os.path.exists(trimmed_reads_dir):
    raise FileNotFoundError(f"Error: trimmed_reads directory does not exist at {trimmed_reads_dir}")

if not os.path.exists(ribofile):
    raise FileNotFoundError(f"Error: human_rRNAs.fasta does not exist at {ribofile}")

if not os.path.exists(sort_dir):
    os.mkdir(sort_dir)
    os.chown(sort_dir, PUID, PGID)

replicates = get_filenames(trimmed_reads_dir)

for key in replicates:
    subprocess.run(["sortmerna",
                    "-ref",
                    ribofile,
                    "-reads",
                    f"{trimmed_reads_dir}/{replicates[key][0]}",
                    "-reads",
                    f"{trimmed_reads_dir}/{replicates[key][1]}",
                    "-aligned",
                    f"{sort_dir}/{key}_rRNA", # label for RNA files
                    "-other",
                    f"{sort_dir}/{key}_nonrRNA", # label for nonrRNA files
                    "--paired_in", # if either read aligns to rRNA, discard in aligned file
                    "--out2", # write non-rRNA neads to 2 files
                    "-fastx",
                    "--threads",
                    "4",
                    "-workdir",
                    f"{sort_dir}/tmp"])
    
    shutil.rmtree(f"{sort_dir}/tmp/kvdb")

shutil.rmtree(f"{sort_dir}/tmp")

# Rename files to match the naming convention used in the rest of the pipeline

for dirpath, dirnames, filenames in os.walk(sort_dir):
    for filename in filenames:
        if "fwd" in filename:
            new_name = filename.replace("fwd", "R1")
            shutil.move(os.path.join(dirpath, filename), os.path.join(dirpath, new_name))
        elif "rev" in filename:
            new_name = filename.replace("rev", "R2")
            shutil.move(os.path.join(dirpath, filename), os.path.join(dirpath, new_name))
        else:
            continue


## Alignment

### Make `STAR` index

Use concatenated WSN/FFLUC and hg38 genome to form a hybrid index for alignment. Use homemade WSN_annotated.gtf for flu and FFLUC spike in control

In [ ]:
cat genome/GCF_000001405.40_GRCh38.p14_genomic.fasta genome/WSN_Mehle.fasta > hybrid_genome.fasta
cat genome/genomic.gtf genome/WSN_annotated.gtf > genome/hybrid_annotated.gtf
STAR --runThreadN 4 --runMode genomeGenerate --genomeDir genome/star_index --genomeFastaFiles genome/hybrid_genome.fasta --sjdbGTFfile genome/hybrid_annotated.gtf --sjdbOverhang 100 --limitGenomeGenerateRAM 30000000000

### Align to hybrid index

In [5]:
# %load run_star_pairedend.py
import os
import shutil
import subprocess
from filename_utils import get_filenames

# This script runs star on paired end R1 R2 reads after ribodepletion

__name__ = "run_star_pairedend"
__author__ = "Thomas"
__date__ = "2024-05-07"

basedir=os.path.abspath(os.path.dirname(__file__))
PUID = os.getuid()
PGID = os.getgid()
reads_dir=f"{basedir}/ribodepleted_reads"

if not os.path.exists(reads_dir):
    raise FileNotFoundError(f"Error: ribodepleted_reads directory does not exist at {reads_dir}")

star_dir = f"{basedir}/star_alignment"
if not os.path.exists(star_dir):
    os.mkdir(star_dir)
    os.chown(star_dir, PUID, PGID)

star_index_dir = f"{basedir}/genome/star_index"
if not os.path.exists(star_index_dir):
    raise FileNotFoundError(f"Error: STAR index directory does not exist at {star_index_dir}")

reads = get_filenames(reads_dir)

for key in reads:
    if len(reads[key]) != 2:
        raise ValueError(f"Error: {key} does not have exactly 2 read files")

# Run star alignment on each sample in paired-end mode
for key in reads:
    if "_nonrRNA" in key:
        keyname = key.replace("_nonrRNA", "")
        subprocess.run(["STAR",
                        "--runThreadN",
                        "4",
                        "--genomeDir",
                        star_index_dir,
                        "--readFilesIn",
                        f"{reads_dir}/{reads[key][0]}",
                        f"{reads_dir}/{reads[key][1]}",
                        "--outFileNamePrefix",
                        f"{star_dir}/{keyname}/{keyname}_",
                        "--quantMode",
                        "TranscriptomeSAM",
                        "--genomeLoad",
                        "LoadAndKeep",
                        "--outReadsUnmapped",
                        "Fastx",
                        "--readFilesCommand",
                        "zcat"])
        # Sort and index bam files
        subprocess.run(["samtools",
                        "view",
                        f"{star_dir}/{keyname}/{keyname}_Aligned.out.sam",
                        "-o",
                        f"{star_dir}/{keyname}/{keyname}_aligned.bam"])
        os.remove(f"{star_dir}/{keyname}/{keyname}_Aligned.out.sam")
        subprocess.run(["samtools",
                        "sort",
                        f"{star_dir}/{keyname}/{keyname}_Aligned.toTranscriptome.out.bam",
                        "-o",
                        f"{star_dir}/{keyname}/{keyname}_Aligned.toTranscriptome.sorted.bam",
                        "-@",
                        "4"])
        subprocess.run(["samtools",
                        "sort",
                        f"{star_dir}/{keyname}/{keyname}_aligned.bam",
                        "-o",
                        f"{star_dir}/{keyname}/{keyname}_coord_sorted.bam",
                        "-@",# for dirpath, dirnames, filenames in os.walk(reads_dir):
                        "4"])
        subprocess.run(["samtools",
                        "index",
                        f"{star_dir}/{keyname}/{keyname}_Aligned.toTranscriptome.sorted.bam",
                        "-@",
                        "4"])
        subprocess.run(["samtools",
                        "index",
                        f"{star_dir}/{keyname}/{keyname}_coord_sorted.bam",
                        "-@",
                        "4"])

# Clean up
if os.path.exists(f"{star_dir}/exit"):
    subprocess.run(["STAR",
                    "--genomeLoad", 
                    "Remove",
                    "--genomeDir",
                    star_index_dir,
                    "--outFileNamePrefix",
                    f"{star_dir}/exit/exit"])
    shutil.rmtree(f"{star_dir}/exit")
        

## Quantify Reads directly from STAR output

## Quantify Reads- Alignment independent (bypass `star`)

Now we need to quantify reads before counting them. `DeSeq2` can take counts from `salmon` (in fact, this is the official recommendation in the docs). In `dusp_11_clip-seq` I discussed quantification methods including simple gene counts and more sophisticated transcript counts. We'll use the genome alignments to inform transcript quantification (and maybe test alignment-independent quantification once this approach is done) in preparation to pass those results to `deseq2`.

`Salmon` needs files aligned to the transcriptome, luckily `star` can output this. It first aligns to genome and them maps these alignments to the transcriptome, but still preserves any novel reads that don't map the provided transcriptome. (Note, this is different than the clip-seq approach because we simply wanted to find peaks in those files, not quantify reads.) However, this is an alignment-dependent approach.

`Salmon` can quantify in an alignment-independent manner, and this method seems to be more accurate than alignment-dependent methods, counterintuitively. This is because using alignment-dependent methods (cufflinks, HTseq, FeatureCounts, maybe salmon in alignment-dependent mode) can vastly underestimate abundance from reads with >90% sequence similarity.

Kind of confusingly, while alignment-independent mapping is superior for quantification, these counts are more accurate when mapped to the genome instead of the transcriptome. This is because transcripts can have conserved UTRs and sequences as well as different spliceoforms. So the best approach here I think is to quantify in an alignment-independent manner and collapse these counts down to the gene level. This is recommended here.

To accomplish this, I'll use salmon to map reads to transcripts without aligning, and then collapse these into counts per gene before passing to `deseq2`. This makes `star` alignment obsolete. We need a transcript index for our hybrid genome for human/WSN transcripts. This is tricky. `gffread` can take a gff file and a genome file and extract transcripts from the genome sequences using the info in the gff file, outputting a transcripts.fasta file. Simply combining the gff files and the genome files results in issues with `gffread` only extracting either human or WSN sequences for some reason. To get around this I ran `gffread` on the WSN genome using the WSN gff and the human genome using the human gff and then combined those. This should work with the combined human/WSN genome that I made for `star` to make indexes for `salmon`.

### Make `salmon` index

Now we need to index the transcriptome (WSN and human) so that salmon can map to it. First use `gffread` to use the gff files I downloaded/created for both genomes to extract only the transcripts.

In [ ]:
gffread -w genome/WSN_transcripts.fasta -g genome/WSN_Mehle.fasta genome/WSN_annotated.gtf
gffread -w genome/human_transcripts.fasta -g genome/GCF_000001405.40_GRCh38.p14_genomic.fasta genome/genomic.gtf
cat genome/WSN_transcripts.fasta genome/human_transcripts.fasta > genome/hybrid_transcripts.fasta

Note: concatenating the gff files and genome files and then running gffread results in either WSN transcripts only or human transcripts only. Probably a formatting issue with homemade gff file. Instead, extract transcripts individually and concatenate them together.

Also, WSN_transcripts.fasta contains HA and NA gene segments for some reason. Manually remove them so they don't break downstream processing and figure out why gffread is including them later.

Now we can make the `salmon` index using the transcripts and masking the genome for more accurate mapping and automatic filtering of DNA contamination. 

In [ ]:
# First make a list of genomic decoys
grep "^>" >(cat genome/hybrid_genome.fasta) | cut -d " " -f 1 > genome/decoys.txt
sed -i.bak -e 's/>//g' genome/decoys.txt

# Make a gentrome file of transcriptome FIRST followed by genome to form the index
cat genome/hybrid_transcripts.fasta genome/hybrid_genome.fasta > genome/hybrid_gentrome.fasta

# Make salmon index
salmon index -t genome/hybrid_gentrome.fasta -d genome/decoys.txt -p 4 -i genome/salmon_index

### Run `Salmon`

In [6]:
# %load find_salmon_replicates.py
#! /usr/bin/env python3

import os
import sys
import subprocess
from filename_utils import get_filenames

__name__ = "find_salmon_replicates"
__author__ = "Thomas"
__date__ = "2024-05-07"

# Use this file to find all replicates denoted by _R(rep number)_ for each sample
# and run Salmon quantification on them. This script will output a directory
# containing the Salmon output files for each sample with replicates combined.

# Define the path to the directory containing the Salmon output files
basedir=os.path.abspath(os.path.dirname(__file__))
PUID = os.getuid()
PGID = os.getgid()
salmon_dir = f"{basedir}/salmon_quantification"
reads_dir = f"{basedir}/ribodepleted_reads/"
salmon_index_dir = f"{basedir}/genome/salmon_index/"

if not os.path.exists(salmon_dir):
    os.mkdir(salmon_dir)
    os.chown(salmon_dir, PUID, PGID)

if not os.path.exists(reads_dir):
    sys.exit("Error: ribodepleted_reads directory does not exist")

if not os.path.exists(salmon_index_dir):
    print("Error: Salmon index directory does not exist")
    print("attempting salmon index")
    subprocess.run(["salmon", 
                    "index", 
                    "-t",
                    "genome/hybrid_gentrome.fasta",
                    "-d", 
                    "genome/decoys.txt",
                    "-p",
                    "4", 
                    "-i", 
                    salmon_index_dir])


# Find replicate files in reads_dir and run salmon quantification on them, 
# combining the replicates for each sample

replicates = get_filenames(reads_dir)

for key in replicates:
    if "_nonrRNA" in key:
        subprocess.run(["salmon",
                        "quant",
                        "-i",
                        salmon_index_dir,
                        "-l",
                        "ISF", # This might not be correct or consistent, check log files
                        # f"<(cat {reads_dir + replicates[key][0]} {reads_dir + replicates[key][1]})"
                        "-1",
                        reads_dir + replicates[key][0],
                        "-2",
                        reads_dir + replicates[key][1],
                        "--validateMappings",
                        "-p",
                        "4",
                        "--seqBias",
                        "--gcBias",
                        "--reduceGCMemory",
                        "--writeUnmappedNames",
                        "-o",
                        salmon_dir + "/" + key.strip("_nonrRNA")])
        
# Rename the output files to include the sample name

for dirpath, dirnames, filenames in os.walk(salmon_dir):
    for file in filenames:
        if "quant.sf" in file:
            sample_name = os.path.basename(dirpath)
            os.rename((os.path.join(dirpath, file)),(os.path.join(dirpath, sample_name + "_quant.sf")))

Salmon returns really low mapping percentages from running on an hg38 index (NOTE: this is still true after myco/ribodepletion, but less so). Maybe this is because many of our hits are mapping to unannotated transcripts (e.g Pol III stuff)? To deal with this, lets assemble a transcriptome *de novo* using reads that we obtained from star. This is called genome-guided assembly and can be done using `Trinity`.

## Use `Trinity` to generate a transcriptome *de novo*

[Trinity](https://www.nature.com/articles/nbt.1883) can assmeble a transcriptome using RNA-seq reads and nothing else. This is really helpful if you don't have a reference genome. It can also assemble a transcriptome *after* aligning to the genome using something like `star`. This is useful if you do have a reference genome but are looking for unannotated transcripts as genome guided assembly may be [more accurate](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-017-1911-6).

To do this, first concatenate all .bam files from star_alignment into one mega bam file for trinity. **Triniy *requires* that you create a single assembly from all samples and then quantify each sample individually againsta it**.

In [8]:
# %load merge_star_bam.py
import os
import subprocess

PUID = os.getuid()
PGID = os.getgid()

if not os.path.exists("trinity"):
    os.makedirs("trinity")
    os.chown("trinity", PUID, PGID)

if not os.path.exists("star_alignment"):
    raise FileNotFoundError("Star alignment directory not at './star_alignment'")

# Get a list of bam files from star_alignment
bam_files = []
for dirpath, dirnames, filenames in os.walk("star_alignment"):
    for filename in filenames:
        if "_aligned.bam" in filename:
            bam_files.append(os.path.join(dirpath, filename))

# Merge bam files
subprocess.run(["samtools",
                "merge",
                "-o",
                "trinity/merged.bam",
                *bam_files
                ])

# Sort bam file by coordinate
subprocess.run(["samtools",
                "sort",
                "-o",
                "trinity/merged_sorted.bam",
                "trinity/merged.bam",
                "-@",
                "4"])

And the run trinity on the large sorted .bam file.

In [9]:
# %load run_trinity.sh
#! /bin/bash

# genome guided trinity on merged bam file

Trinity --genome_guided_bam trinity/merged_sorted.bam \
    --genome_guided_max_intron 1000000 \
    --max_memory 30G \
    --CPU 4 \
    --output trinity/

This takes 36 hours and generates a directory with a `Trinity-GG.fasta` file containing assembled transcripts. Let's run salmon quantification of both raw reads and aligned reads against this new transcriptome as an index.

### Quantify using salmon against the trinity generated transcriptome.

In [10]:
# %load salmon_trinity.py
#! /usr/bin/env python3
import os
import sys
import subprocess
from filename_utils import get_filenames

__name__ = "salmon_trinity"
__author__ = "Thomas"
__date__ = "2024-05-013"

salmon_index_dir = "trinity/salmon_index"
basedir = os.path.abspath(os.path.dirname(__file__))
PUID = os.getuid()
PGID = os.getgid()
salmon_dir = f"{basedir}/salmon_quantification_trinity"
reads_dir = f"{basedir}/ribodepleted_reads"
trinity_file = f"{basedir}/trinity/Trinity-GG.fasta"
genome_dir = f"{basedir}/genome"

# Use this script to run Salmon quantification on the Trinity assembled transcripts

# Check for trinity directory
if not os.path.exists("trinity"):
    raise FileNotFoundError("Error: trinity directory does not exist at ./trinity!")
else:
    os.chown("trinity", PUID, PGID)

# Check for Trinity-GG.fasta
if not os.path.exists("trinity/Trinity-GG.fasta"):
    raise FileNotFoundError("Error: Trinity-GG.fasta not found in ./trinity!")

# Check if trinity specific salmon index directory exists
# If not, create it using trinity_decoys.txt
# If trinity_decoys.txt does not exist, create it
if not os.path.exists(salmon_index_dir):
    print("trinity/salmon_index directory does not exist!")
    print("Attempting to create trinity/salmon_index directory...")
    os.mkdir(salmon_index_dir)
    os.chown(salmon_index_dir, PUID, PGID)
   
    # Check for hybrid decoys.txt from hg38 and WSN genomes
    if not os.path.exists(f"{genome_dir}/decoys.txt"):
        print("decoys.txt not found, creating...")
        result = subprocess.run(
            [
                "grep",
                "'^>'",
                f"<(cat {genome_dir}/hybrid_genome.fasta) | cut -d ' ' -f 1 > {genome_dir}/decoys.txt",
            ]
        )
        if result.returncode != 0:
            raise RuntimeError("Error: Failed to create trinity_decoys.txt!")
        result = subprocess.run(
            ["sed", "-i.bak", "-e", "s/>//g", f"{genome_dir}/decoys.txt"]
        )
        if result.returncode != 0:
            raise RuntimeError("Error: Failed to edit trinity_decoys.txt!")
    
    if not os.path.exists(trinity_file):
        raise FileNotFoundError("Error: Trinity-GG.fasta not found!")
    
    # Create hybrid gentrome out of trinity and hybrid genome from hg38 and WSN
    if not os.path.exists(f"{genome_dir}/hybrid_genome.fasta"):
        raise FileNotFoundError("Error: hybrid_genome.fasta not found!")
    
    if not os.path.exists("trinity/trinity_gentrome.fasta"):
        print("trinity_gentrome.fasta not found, creating...")
        result = subprocess.run(
            f"cat {trinity_file} {genome_dir}/hybrid_genome.fasta > trinity/trinity_gentrome.fasta",
            shell=True,
        )
        if result.returncode != 0:
            raise RuntimeError("Error: Failed to create trinity_gentrome.fasta!")
    
    result = subprocess.run(
        [
            "salmon",
            "index",
            "-t",
            "trinity/trinity_gentrome.fasta",
            "-d",
            f"{genome_dir}/decoys.txt",
            "-i",
            salmon_index_dir,
            "-p",
            "4",
        ]
    )
    if result.returncode != 0:
        raise RuntimeError("Error: Failed to create trinity salmon index!")

if not os.path.exists(salmon_dir):
    os.mkdir(salmon_dir)
    os.chown(salmon_dir, PUID, PGID)

# Find all the fastq files in the ribodepleted directory
files = get_filenames(reads_dir)

# Run salmon quantification aligning to Trinity assembled transcripts
for key in files:
    if "_nonrRNA" in key:
        result = subprocess.run(
            [
                "salmon",
                "quant",
                "-i",
                salmon_index_dir,
                "-l",
                "ISF",
                "-1",
                f"{reads_dir}/{files[key][0]}",
                "-2",
                f"{reads_dir}/{files[key][1]}",
                "-p",
                "4",
                "-o",
                f"{salmon_dir}/{key}",
            ]
        )
        if result.returncode != 0:
            raise RuntimeError(f"Error: Failed to run salmon quantification on {key}!")
